In [1]:
import numpy as np
import nltk

from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup

In [2]:
wordnet_lemmatizer = WordNetLemmatizer()

In [3]:
# from http://www.lextek.com/manuals/onix/stopwords1.html
# could also use english stopwords from nltk.corpus.stopwords
stopwords = set(w.rstrip() for w in open('../data/stopwords.txt'))

In [4]:
positive_reviews = BeautifulSoup(open('../data/electronics/positive.review').read())
positive_reviews = positive_reviews.findAll('review_text')

/Users/taylorperkins/Desktop/projects/nlp-101/venv/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [5]:
negative_reviews = BeautifulSoup(open('../data/electronics/negative.review').read())
negative_reviews = negative_reviews.findAll('review_text')

/Users/taylorperkins/Desktop/projects/nlp-101/venv/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [6]:
np.random.shuffle(positive_reviews)
positive_reviews = positive_reviews[:len(negative_reviews)]

In [7]:
def my_tokenizer(s):
    tokens = list()
    
    for t in nltk.tokenize.word_tokenize(s.lower()):
        if len(t) > 2:
            lem = wordnet_lemmatizer.lemmatize(t)
            if lem not in stopwords:
                tokens.append(lem)    

    return tokens

In [8]:
word_index_map = {}
current_index = 0

positive_tokenized = list()
negative_tokenized = list()

In [9]:
word_index_map = {}
current_index = 0

for review in positive_reviews:
    tokens = my_tokenizer(review.text)
    positive_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

for review in negative_reviews:
    tokens = my_tokenizer(review.text)
    negative_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

In [10]:
word_index_map

{'this': 0,
 'phone': 1,
 'battery': 2,
 'life': 3,
 'awesome': 4,
 'range': 5,
 'interference': 6,
 'feature': 7,
 'subtstanial': 8,
 'quality': 9,
 'amazing': 10,
 'ha': 11,
 'uniden': 12,
 'cordless': 13,
 'substantial': 14,
 'upgrade': 15,
 'tripp': 16,
 'lite': 17,
 'charge': 18,
 'powerbook': 19,
 'digital': 20,
 'camera': 21,
 'cross': 22,
 'country': 23,
 'trip': 24,
 'grad': 25,
 'school': 26,
 'wa': 27,
 'outlet': 28,
 'using': 29,
 'monster': 30,
 'cable': 31,
 'home': 32,
 'theater': 33,
 'powercenter': 34,
 'hts800': 35,
 'remove': 36,
 'line': 37,
 'noise': 38,
 'electrical': 39,
 'current': 40,
 'high-end': 41,
 'tube': 42,
 'pre-amp': 43,
 'removed': 44,
 'beautifully': 45,
 'however': 46,
 'creates': 47,
 'high-pitched': 48,
 'tone': 49,
 'device': 50,
 'directly': 51,
 'you': 52,
 'planning': 53,
 'product': 54,
 'sound': 55,
 'system': 56,
 'recommend': 57,
 'placing': 58,
 'location': 59,
 'acoustically': 60,
 'isolated': 61,
 'received': 62,
 'speaker': 63,
 'repla

In [11]:
def tokens_to_vector(tokens, label):
    # really x and y
    x = np.zeros(len(word_index_map) + 1)
    for t in tokens:
        # get the index from the map
        i = word_index_map[t]
        x[i] += 1
        
    # so they sum up to one.
    x = x / x.sum()
    x[-1] = label
    
    return x

In [12]:
N = len(negative_tokenized) + len(positive_tokenized)

In [13]:
data = np.zeros((N, len(word_index_map) + 1))
i = 0

for tokens in  positive_tokenized:
    data[i,:] = tokens_to_vector(tokens, 1)
    i += 1
    
for tokens in negative_tokenized:
    data[i,:] = tokens_to_vector(tokens, 0)
    i += 1

In [14]:
np.random.shuffle(data)

X = data[:, :-1]
Y = data[:, -1]

Xtrain  = X[:-100,]
Ytrain  = Y[:-100,]
Xtest = X[-100:,]
Ytest = Y[-100:,]

In [15]:
model = LogisticRegression()
model.fit(Xtrain, Ytrain)
print(f"Classification score: {model.score(Xtest, Ytest)}")

Classification score: 0.71


In [16]:
threshold = 0.5
for word, index in word_index_map.items():
    weight = model.coef_[0][index]
    if weight > threshold or weight < -threshold:
        print(word, weight)

feature 0.5100680936574866
quality 1.4250904515833998
ha 0.7604657368422274
wa -1.596076789349413
using 0.5965113136939229
cable 0.5894291345245279
home 0.5614979379808615
you 1.0946728416150675
sound 1.1319793174896997
recommend 0.6047354639087335
speaker 0.8933190642239174
happy 0.6640674711128797
highly 0.917507575685576
doe -1.1724493545281014
time -0.714688535619022
excellent 1.3142435612266823
little 0.9040712996065262
buy -0.9067221172985255
fit 0.5151798691731115
space 0.5111180659355755
lot 0.7491982602537445
n't -2.0917254615398106
look 0.6023473761150189
picture 0.5840320446234702
pretty 0.7055173817237375
support -0.8490011369336031
static -0.5241633790759991
unit -0.6052588941004251
waste -1.0067199864053666
company -0.5438240281246427
've 0.7173142592076108
laptop 0.505571738975049
comfortable 0.6159799623802469
try -0.6000793560646981
then -1.0415653445912825
perfect 1.0085303186982506
month -0.8047732265416299
love 1.2134045241684812
memory 0.9283135407247141
returned -